In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output


config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'

model = cv2.dnn_DetectionModel(frozen_model, config_file)

classLabels = []
file_name = 'labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')


In [2]:
print(classLabels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [3]:
model.setInputSize(320, 320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5, 127.5, 127.5))
model.setInputSwapRB(True)

< cv2.dnn.Model 00000219B8C23170>

In [5]:
# test_video.mp4
# video
import math
cap = cv2.VideoCapture("test.mp4")
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Can’t open the video")

font_scale = 1
font = cv2.FONT_HERSHEY_PLAIN

while True:
    ret, frame = cap.read()
    objects = {}

    frame_height, frame_width, _ = frame.shape
    frame_centerX, frame_centerY = frame_width//2, frame_height//2
    reduce = 25
    range_xs, range_xe, range_ys, range_ye = frame_centerX - (frame_width*reduce/100), frame_centerX + (
        frame_width*reduce/100), frame_centerY - (frame_height*reduce/100), frame_centerY + (frame_height*reduce/100)
    range_box = np.array([range_xs, range_ys, range_xe - range_xs, range_ye - range_ys
                          ], dtype=int)
    ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)

    # print(ClassIndex)

    if (len(ClassIndex) != 0):
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if (ClassInd <= 80):
                label, ratio = classLabels[ClassInd-1], boxes[2]/boxes[3]
                object_centerX, object_centerY = (math.floor(
                    boxes[0]+(boxes[2]/2)), math.floor(boxes[1]+(boxes[3]/2)))
                cv2.rectangle(frame, range_box, (255, 0, 0), 2)
                cv2.putText(frame, label, (object_centerX,object_centerY),
                            font, fontScale=font_scale, color=(0, 255, 255), thickness=3)
                if (range_xs<object_centerX and object_centerX < range_xe) and (range_xs<object_centerY and object_centerY < range_xe):
                    cv2.circle(frame, (object_centerX, object_centerY),
                           3, color=(0, 0, 255), thickness=4)
                    if label in objects.keys():
                        _bbox = objects[label]
                        if _bbox[2] < boxes[2] or _bbox[3] < boxes[3]:
                            objects[label] = boxes
                    else:
                        objects[label] = boxes
                
        for box in objects.values():
            print(box)
            cv2.circle(frame, (box[0] + box[2]//2,box[1] + box[3]//2),
                    3, color=(0, 255, 0), thickness=4)
        print(objects)
        clear_output(wait=True)  


    cv2.imshow('Object Detection', frame)

    # cv2.waitKey(0)

    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[337 455 197 148]
[862 416 156 152]
[321 426  43  67]
{'car': array([337, 455, 197, 148]), 'truck': array([862, 416, 156, 152]), 'person': array([321, 426,  43,  67])}
